In [17]:
import pandas as pd
%matplotlib inline
import urllib
import vzta
import vzta.feature_extractors as fe
import vzta.algorithms as algorithm



In [36]:
from spacy.symbols import *
from itertools import takewhile
#text='your automated system is not answering my questions!!! at\u0026t is looking better than you'.decode('unicode-escape')
text="can i set up a date to pay the bill in full i won\u0027t be able to do it until after the first of the month or on the 1st so basically i need an extension"
text="can i change Verizon Wireless plans during my contract term?"
##text ="make a payment arrangement"


def printDoc(text):
    doc= vzta.nlp.getDoc(text,parse=True)
    dlen =len(doc)
    for d in doc:
        print( d.orth_, d.pos_, d.dep_, d.head.orth_)

        
#printDoc(text)

OBJECTS=[dobj, iobj, pobj]
def dedup(l):
    result=[]
    for i in l:
        if i in result:
            continue
        result.append(i)
    return result

def isNegated(v):
    for c in v.children:
        if c.dep==neg:
            return True
    return False

def getPart(v):
    for c in v.rights:
        if c.pos==PART or c.dep==acomp:
            return c
    return None

def get_span_for_compound_noun(noun):

    min_i = noun.i - sum(1 for _ in takewhile(lambda x: x.dep_ in ('compound','amod'),
                                              reversed(list(noun.lefts))))
    return (min_i, noun.i)

def getPreps(deps):
    objs = []
    for dep in deps:
        if dep.pos == ADP and dep.dep == prep:
            objs.extend(
                [tok for tok in dep.rights if tok.dep in OBJECTS or tok.pos == PRON])
    return objs



np_labels = set([nsubj, nsubjpass,dobj, iobj, pobj,attr,dep])
#np_labels = set([dobj, iobj, pobj])
def iter_nps(doc):
    for word in doc:
        if word.dep in np_labels:
            yield [word.head,word.subtree,word.dep]

def isq(doc):
    first_token=doc[0].text.lower()
    last_token=doc[-1].text.lower()
    if first_token in ['when','what','how','why','where']:
        if first_token=='how':
            if doc[1].text=='much':
                first_token ='how much'
            elif doc[1].text=='many':
                first_token ='how many'
        return first_token+'?'
    if first_token in ['am','do','does','will', 'is', 'should']:
        return '?'
    if last_token=='?':
        return '?'
    return None
def intent(doc):
    result=[]
    for sent in doc.sents:
        itr= iter_nps(sent)
        intents=[]
        negations=False
        word=None
        sent_result=[]
        for p in itr:
            intent=[t for t in p[1] if t.pos in [VERB,NOUN,PROPN,ADJ,ADV,NUM,X] and not t.text.lower() in ['where','so', 'how','when', 'what','much','many','that','my', 'your','I',"'m"]]
            word= p[0]
            if intent: #and word.pos==VERB:  #and and word.lemma_!='be'
                preps = getPreps(word.children)
                intent =intent +[pr for pr in preps]
                action= []
                negated=False
                if word.pos==VERB:
                    negated = isNegated(word)
                    part = getPart(word)
                    action.append(word)
                    if part:
                        action.append(part)
                else:
                    if word.dep==prep:
                        if word.head.pos==VERB:
                            action.append(word.head)
                            negated = isNegated(word.head)
                        elif word.head.pos==ADJ:
                            action.append(word.head)
                            negated = isNegated(word.head)
                            if not negated and word.head.dep !=root:
                                negated = isNegated(word.head.head)
                        else:
                            min_i,max_i = get_span_for_compound_noun(word.head)
                            spn=doc[min_i:max_i+1]
                            action.extend([a for a in spn])
                    elif word.dep==prt:
                        action.extend([word.head,word])
                    elif word.pos in(NOUN,PROPN):
                        min_i,max_i=get_span_for_compound_noun(word)
                        spn=doc[min_i:max_i+1]
                        action.extend([a for a in spn])
                    else:
                        action.append(word)
                sent_result.append([negated,action, intent])
        
        if sent_result:
            sent_result=combineSentResult(sent_result, doc)
            sent_data=[]
            for sr in sent_result:
                action = ' '.join([ a.text for a in sr[1]])
                if sr[0]:
                    action ='!'+ action
                intent= action+ ':'+' '.join([ a.text for a in sr[2]])
                sent_data.append(intent)
            q = isq(sent)
            r =[sent.text, ','.join(sent_data), q]
            result.append(r)    
    return result
 
def combineResult( r1, r2,doc):
    if not r1 or not r2:
        return None
    
    r1hasVerb=  len ([v for v in r1[1] if v.pos==VERB])>0
    r2hasVerb = len ([v for v in r2[1] if v.pos==VERB])>0
    
    if r1hasVerb and not r2hasVerb:
         return r1
    
    if r2hasVerb and not r1hasVerb:
         return r2
    
    combine=0
    for a2 in r2[1]:
        if a2 in r1[1]:
            combine=2
            break
            
    if combine==0:
        for i2 in r2[2]:
            if i2 in r1[2]:
                combine=1
                break
    
    if combine==0:
        commonVerbs = ["is","are","'s","'re","am","'m"]

        r1CommonVerbs =len ([v for v in r1[1] if v.text.lower() in commonVerbs ])>0
        r2CommonVerbs =len ([v for v in r2[1] if v.text.lower() in commonVerbs ])>0
        if r2CommonVerbs and not r1CommonVerbs:
            combine=1
        #elif r1CommonVerbs and not r2CommonVerbs:
            #combine=3
    
    if combine==0:
        l = doc.__len__() - 1
        for t in r1[2]:
            if t.i  < l  and  t.nbor(1) in r2[2]:
                combine=2
                break
       
    
    if combine==1:
        return r1
    elif combine==2:
        newActions = sorted(dedup(r1[1]+r2[1]), key=lambda x: x.i)
        newIntents = sorted(dedup(r1[2]+r2[2]),key=lambda x:x.i)
        negated = r1[0] or r2[0]
        return [negated,newActions,newIntents]
    elif combine==3:
         return r2
        
    return None
        


def combineSentResult(sent_result,doc):
    if len(sent_result)<2:
        return sent_result
    newResult=[]
    i=0
    l= len(sent_result)
    while i < l:
        combined=None
        r1 =sent_result[i]
        if i+1 < l:
            r2 =sent_result[i+1]
        else:
            newResult.append(r1)
            break;
        combined = combineResult(r1,r2,doc)
        if combined:
            sent_result[i+1]= combined
        else:
            newResult.append(r1)
            
        i=i+1;
    return newResult
                
                
    


def intent_extractor(text):
    txt=text.decode('utf-8').strip()
    #txt = re.sub(r'(?!\\+u00)','',txt)
    #txt=unicode(txt)
    try:
        txt =txt.decode('unicode-escape')
        txt = txt.replace('&','and')

    except:
        txt = txt.replace('\\','')
    doc = vzta.nlp.getDoc(txt,parse=True)
    result=[]
    if len(doc)>3:
        result = intent(doc)
        
    return result


#intent_extractor('why do i have to pay an upgrade fee?. ok. ok. Larry Zimmerman.  ok. we want to get the iphone on 410-000-000. mine is 410.000.000. i am Larry. i was looking at do a contract and pay up front too. it looks like $299. and equipment protection')
        


def getIntents(text):
    res=[]
    result={}
    intents= intent_extractor(text)
    for i in intents:
        result['sent']= i[0].text
        if r[1]:
                result['negation']='!'
        else:
                result['negation']=''
        actions = i[2]
        result['action'] = ' '.join(dedup([a.text for a in actions]))
        result['subject']=' '.join(dedup([a.text for a in i[3]]))
        q= i[4]
        if q:
                result['Q']=q
        else:
               result['Q'] =''
        res.append(result)
    return res

intent_extractor(text)
     

[[u'can i change Verizon Wireless plans during my contract term?',
  u'change:Verizon Wireless plans contract term',
  '?']]

In [30]:
search_file=r"C:\Users\v617346\Documents\TA\data\search.txt"
search_corpus = vzta.corpus.Corpus(search_file)
search_corpus=search_corpus[0]
original=[]
sents=[]
negations=[]
verbs=[]
nverbs=[]
intents=[]
Qs=[]

#[sent,negated,verb,intent,q]

for text in search_corpus:
    result= intent_extractor(text)
    if result:
        for r in result:
            original.append(text)
            sents.append(r[0])
            intents.append(r[1])
            q= r[2]
            if q:
                Qs.append(q)
            else:
                Qs.append('')
            
data = {'Original': original,'Sentence': sents, 'intent':intents, 'Question':Qs}
dframe = pd.DataFrame(data, columns=data.keys())

C:\Users\v617346\Documents\TA\data\search.txt


In [32]:
dframe.to_excel(r'C:\Users\v617346\Documents\TA\intent_mappings13.xlsx', index=False)